In [18]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import datetime
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import math
from numpy.linalg import svd

from sklearn.ensemble import RandomForestClassifier

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from sklearn.externals import joblib


### Loading the initial dataset

In [3]:
path = os.getcwd() + '/../dataset/anonymized_dataset_for_ADM2017/'
print(path + 'student_log_{}.csv'.format(1))
log1 = pd.read_csv(path + 'student_log_{}.csv'.format(1))
initial_col_order = [col.lower() for col in list(log1.columns)]

data = pd.DataFrame()
for i in range(1, 10):
    student_log = pd.read_csv(path + 'student_log_{}.csv'.format(i))
    student_log.columns = [col.lower() for col in student_log]
    data = pd.concat([data, student_log], ignore_index=True)

# reorder everything with the original order, where student id is in the first column
data = data[initial_col_order]
data.shape
data.head()

/Users/yeldosbalgabekov/Desktop/Books and Schools/UoSouthampton/Data Mining/DataMining/Yeldos_Richa/../dataset/anonymized_dataset_for_ADM2017/student_log_1.csv


/Users/yeldosbalgabekov/anaconda2/envs/py35/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,...,confidence(off task),confidence(gaming),res_bored,res_concentrating,res_confused,res_frustrated,res_offtask,res_gaming,ln-1,ln
0,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.838710,0.008522,0.376427,0.320317,0.000000,0.0,0.785585,0.000264,0.13,0.0611904
1,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.600000,0.047821,0.156027,0.995053,0.887452,0.0,0.468252,0.001483,0.0611904,0.21351
2,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.600000,0.047821,0.156027,0.995053,0.887452,0.0,0.468252,0.001483,0.116,0.0333058
3,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.204082,0.343996,0.156027,0.744520,0.000000,0.0,0.108417,0.010665,0.116,0.0333058
4,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.204082,0.343996,0.156027,0.744520,0.000000,0.0,0.108417,0.010665,0.0333058,0.118386


### Transforming the initial dataset and loading Training Set

In [4]:
def to_float(x):
    try:
        if x.isdigit():
            return float(x)
    except:
        if x == '.':
            return 0
        return x

def series_types(series):
    types = []
    for i in set(series):
        if not type(i) in types:
            types.append(type(i))
    return(types)


data['sy assistments usage'] = data['sy assistments usage'].astype('category')
data['skill'] = data['skill'].astype('category')
data['problemtype'] = data['problemtype'].astype('category')
data['ln-1'] = data['ln-1'].apply(lambda x: to_float(x))
data['ln'] = data['ln'].apply(lambda x: to_float(x))
# data.memory_usage(deep=True)
# data.info(memory_usage='deep')

training = pd.read_csv(path + 'training_label.csv').drop(['AveCorrect'], axis=1)
training.columns = [col.lower() for col in training.columns]
training.shape

# merged data set with isSTEM
df = data.merge(training, on='itest_id', how="left")
#df.info()


### Filtering for the actions of the "training" students

In [5]:
actions = df[df.isstem.notnull()]
actions['skill'] = actions.skill.astype('category')
print("training set size: ", training.shape[0], "actions for # of students: ", len(actions.itest_id.unique()))
print("sample size useless for training: ", training.shape[0] - len(actions.itest_id.unique()))

# merged training data set
stem = actions[actions.isstem == 1]
nonstem = actions[actions.isstem == 0]

actions.head()

training set size:  514 actions for # of students:  326
sample size useless for training:  188


/Users/yeldosbalgabekov/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,...,res_concentrating,res_confused,res_frustrated,res_offtask,res_gaming,ln-1,ln,schoolid,mcas,isstem
1056,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.262675,0.060808,0.000000,0.889219,0.005797,0.271000,0.077899,2.0,34.0,0.0
1057,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.696486,0.000000,0.009561,0.108417,0.001483,0.077899,0.225856,2.0,34.0,0.0
1058,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.225856,0.483008,2.0,34.0,0.0
1059,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.483008,0.745290,2.0,34.0,0.0
1060,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.898073,0.000000,0.009561,0.468252,0.001483,0.745290,0.900994,2.0,34.0,0.0


### Testing the cases
Analysing other features

In [6]:
stem, non_stem = actions[actions['isstem']==1], actions[actions['isstem']==0]
balanced = pd.concat([stem, non_stem.iloc[:, :stem.shape[0]]])
balanced['sy assistments usage'] = balanced['sy assistments usage'].cat.codes
balanced['skill'] = balanced['skill'].cat.codes
balanced['problemtype'] = balanced['problemtype'].cat.codes
c1 = pd.get_dummies(balanced['sy assistments usage'], prefix=['yos'])
c2 = pd.get_dummies(balanced['skill'], prefix=['skill'])
c3 = pd.get_dummies(balanced['problemtype'], prefix=['problemtype'])
print(balanced.shape, c1.shape, c2.shape, c3.shape)
cats = pd.concat([c1, c2, c3], axis=1)
cats.shape, balanced.shape
table = pd.concat([balanced.iloc[:, -1], balanced.iloc[:, 0], balanced.iloc[:, 1:-1], cats], axis=1)
table.drop(columns=['sy assistments usage', 'skill', 'problemtype'])
print(table.shape)
table.head()

(177960, 79) (177960, 2) (177960, 93) (177960, 16)
(177960, 190)


,isstem,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,...,['problemtype']_6,['problemtype']_7,['problemtype']_8,['problemtype']_9,['problemtype']_10,['problemtype']_11,['problemtype']_12,['problemtype']_13,['problemtype']_14,['problemtype']_15
72563,1.0,1047,0,0.102428,0.062206,0.304348,506,0.205797,0.689716,0.08071,...,0,0,0,0,0,0,0,0,0,1
72564,1.0,1047,0,0.102428,0.062206,0.304348,506,0.205797,0.689716,0.08071,...,0,0,0,0,0,0,0,0,0,1
72565,1.0,1047,0,0.102428,0.062206,0.304348,506,0.205797,0.689716,0.08071,...,0,0,0,0,0,0,0,0,1,0
72566,1.0,1047,0,0.102428,0.062206,0.304348,506,0.205797,0.689716,0.08071,...,0,0,0,0,0,0,0,0,1,0
72567,1.0,1047,0,0.102428,0.062206,0.304348,506,0.205797,0.689716,0.08071,...,0,0,0,0,0,0,0,0,0,1


#### 50 words

In [14]:
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import normalize
# from scipy.spatial.distance import cdist
# from sklearn import metrics

# #TODO: normalize
# table = table.fillna(0)
# # voc_size = 200

# distortions = []
# models = []

# #50
# k = 50
# kmeanModel = KMeans(n_clusters=k)
# print("Kmodel with {} clusters created".format(k))
# kmeanModel.fit(table)
# print("Model is trained")
# distortions.append(sum(np.min(cdist(table, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / table.shape[0])
# print("Distortions calculatd")
# models.append(kmeanModel)

Kmodel with 50 clusters created
Model is trained
Distortions calculatd


In [22]:
# joblib.dump(models[0], '50wordskmeans.pkl')

['50wordskmeans.pkl']

#### 200 Words

In [15]:
# #200
# k = 200
# kmeanModel = KMeans(n_clusters=k)
# print("Kmodel with {} clusters created".format(k))
# kmeanModel.fit(table)
# print("Model is trained")
# distortions.append(sum(np.min(cdist(table, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / table.shape[0])
# print("Distortions calculatd")
# models.append(kmeanModel)

Kmodel with 200 clusters created
Model is trained
Distortions calculatd


In [21]:
# joblib.dump(models[1], '200wordskmeans.pkl')

['200wordskmeans.pkl']

#### 500 Words

In [17]:
# #500
# k = 500
# kmeanModel = KMeans(n_clusters=k)
# print("Kmodel with {} clusters created".format(k))
# kmeanModel.fit(table)
# print("Model is trained")
# distortions.append(sum(np.min(cdist(table, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / table.shape[0])
# print("Distortions calculatd")
# models.append(kmeanModel)

Kmodel with 500 clusters created
Model is trained
Distortions calculatd


In [20]:
# joblib.dump(models[2], '500wordskmeans.pkl')

['500wordskmeans.pkl']

#### 1000 Words

In [32]:
# #1000
# k = 1000
# kmeanModel = KMeans(n_clusters=k)
# print("Kmodel with {} clusters created".format(k))
# kmeanModel.fit(table)
# print("Model is trained")
# distortions.append(sum(np.min(cdist(table, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / table.shape[0])
# print("Distortions calculatd")
# models.append(kmeanModel)

Kmodel with 1000 clusters created
Model is trained
Distortions calculatd


In [34]:
# joblib.dump(models[3], '1000wordskmeans.pkl')

['1000wordskmeans.pkl']

In [35]:
models = [joblib.load('50wordskmeans.pkl'), 
          joblib.load('200wordskmeans.pkl'), 
          joblib.load('500wordskmeans.pkl'), 
          joblib.load('1000wordskmeans.pkl')]


In [36]:
models

[KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=50, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=200, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=500, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0),
 KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=1000, n_init=10, n_jobs=1, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0)]

In [37]:
names = ["50BoWords.csv", "200BoWords.csv", "500BoWords.csv", "1000BoWords.csv"]
# for idx in range(len(names)):
for idx in range(4):
    kmeans = models[idx]
    words = kmeans.labels_.reshape(table.shape[0], 1)
    tmp = table.iloc[:, :2]
    tmp = tmp.reset_index().iloc[:, 1:]
    tmp2 = pd.DataFrame(words).reset_index().iloc[:,-1]
    with_words = pd.concat([tmp, tmp2], axis=1)
    with_words.columns = ['isstem', 'itest_id', 'words']
    with_words = with_words.groupby(['itest_id', 'words'], as_index=False).count()
    histog_features = with_words.pivot(index='itest_id', columns='words', values='isstem').fillna(0)
    histog_features = pd.DataFrame(histog_features.to_records())
    histog_features = histog_features.merge(tmp[['itest_id', 'isstem']], on='itest_id', how='left')
    histog_features = histog_features.drop_duplicates()
    histog_features.to_csv("features/"+names[idx])
    

((253, 202), (73, 202))